# LSP Ti-6Al-4V Depth Stress Scan

This notebook presents an example analysis of residual stresses at P61A.

###### Sample:
100 × 20 × 1 mm thick Ti-6Al-4V sheet with Laser Shock Peening treated surface.

###### Experiment parameters:
Measurements in transmission.
Incident beam 50 × 50 μm, detector 2Θ = 5.274° calibrated using NIST 640f Si powder.

###### Experimental procedure:
1. Orient the sample with surface parallel to the beam at half of the beam height.
2. Rotate the sample by Θ, so that the incident and diffracted beam fall at the same angle.
3. Perform *eu.chi* scan for every *eu.z* position for two *eu.phi* orientations 0° and 90°.
*eu.z* puts the gauge volume at a certain depth in the sample, and for each depth a $sin^2(\psi)$ scan
is performed for $\varphi \in \{0^{\circ}, 90^{\circ}\}$.
For the purposes of $sin^2(\psi)$ method, *eu.phi* is $\varphi$ and *eu.chi* is $\psi$.
4. Diffraction patterns were analysed using P61A::Viewer.
Peaks exported from the Viewer are the starting point of this analysis.

## Imports

In [11]:
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from uncertainties import unumpy, ufloat

# This has to be the path to P61AToolkit/src/py61a.
# If you are launching the notebook from P61AToolkit/src/notebooks it is correct,
# otherwise change it accordingly.
sys.path.append('../py61a')
from py61a.viewer_utils import read_peaks, valid_peaks, peak_id_str, group_by_motors
from py61a.cryst_utils import bragg, tau, mu
from py61a.stress import Sin2Psi, DeviatoricStresses, MultiWaveLength

%matplotlib inline

## Definitions

In [2]:
# 2Theta angle
tth_ch1 = 5.274
# Main element in the alloy, will be used for absorption calculation
element = 'Ti'
# Diffraction elactic constants prepared using ISODEC https://www.nist.gov/services-resources/software/isodec
dec = pd.read_csv(r'../../data/dec/alphaTi.csv', index_col=None, comment='#')

## $d_0$ measurement preparation
A reference sample was measured to get the $d_0$ values for the diffraction peaks.

In [9]:
# read Viewer output
d0_data = read_peaks(r'../../data/peaks/TiLSP/Ti_1mm_d0_ref.csv')
# create a dataframe for storing d0 values
d0 = pd.DataFrame(columns=['h', 'k', 'l', 'd0', 'd0_std', 'en'])
for peak_id in valid_peaks(d0_data, valid_for='phase'):
    # calculating the d value from the peak center and 2Theta
    d_val = bragg(en=ufloat(d0_data[peak_id]['center'].mean(), d0_data[peak_id]['center'].std()), tth=tth_ch1)['d']
    # adding to the d0 dataframe as a new row
    d0.loc[d0.shape[0]] = {
        **{idx: d0_data[peak_id][idx].mean().astype(int) for idx in ('h', 'k', 'l')},
        'd0': d_val.nominal_value,
        'd0_std': d_val.std_dev,
        'en': d0_data[peak_id]['center'].mean()
    }
# showing results: hkl, measured d0, measured d0 standard deviation, energy of the peak
d0 = d0.sort_values(by='en').reset_index(drop=True)
print(d0)

    h  k  l        d0    d0_std          en
0   1  0  0  2.532843  0.000550   53.197829
1   0  0  2  2.335838  0.000622   57.684555
2   1  0  1  2.227108  0.000479   60.500767
3   1  0  2  1.717717  0.000366   78.442366
4   1  1  0  1.463043  0.000455   92.096946
5   1  0  3  1.327186  0.000373  101.524369
6   2  0  0  1.267117  0.000715  106.337267
7   1  1  2  1.240123  0.000234  108.651924
8   2  0  1  1.222798  0.000458  110.191368
9   0  0  4  1.168465  0.000659  115.315181
10  2  0  2  1.113966  0.000360  120.956806


In [13]:
# Reading the Viewer output. Multiple files can be passed to read_peaks function as a tuple
dd = read_peaks((r'../../data/peaks/TiLSP/phi0Trans_3.csv', r'../../data/peaks/TiLSP/phi90Trans_4.csv'))

# Defining the experiment
dd = group_by_motors(
    dd,
    motors=[
        {'mot_name': 'eu.z', 'atol': 1e-3},
        {'mot_name': 'eu.phi', 'atol': 5., 'values': Sin2Psi.phi_values},
        {'mot_name': 'eu.chi', 'atol': .1, 'max': 90.}
    ])